In [1]:
import pandas as pd

In [ ]:
df = pd.read_parquet("../sourcedata/df_metrics.parquet")
df.sort_values(by=['scene_full_name', 'subject', 'learning_phase'], inplace=True)

,subject,learning_phase,scene_full_name,delta_clr_tot,delta_spd_tot,delta_MAD_tot,count,cleared,speed,MAD_mean,level_full_name,scene
3267,im_sub-01,sub-01_epoch=0-step=2000,1-1-0,-0.909091,-59.380211,NaN,44.0,1.000000,65.546218,NaN,w1l1,0
3847,im_sub-01,sub-01_epoch=0-step=3500,1-1-0,-0.909091,-59.380211,NaN,44.0,1.000000,67.774272,NaN,w1l1,0
3557,im_sub-01,sub-01_epoch=0-step=500,1-1-0,-0.909091,-59.380211,NaN,44.0,1.000000,65.546218,NaN,w1l1,0
4137,im_sub-01,sub-01_epoch=0-step=5000,1-1-0,-0.909091,-59.380211,NaN,44.0,1.000000,69.729730,NaN,w1l1,0
2977,im_sub-01,sub-01_epoch=0-step=6500,1-1-0,-0.909091,-59.380211,NaN,44.0,0.090909,6.166008,NaN,w1l1,0
...,...,...,...,...,...,...,...,...,...,...,...,...
11715,sub-05,Late practice,8-3-9,-1.000000,-108.170498,-0.261936,NaN,NaN,NaN,NaN,w8l3,9
12295,sub-06,Early discovery,8-3-9,0.133333,10.194895,-8.961299,6.0,0.666667,69.068186,10.583333,w8l3,9
12585,sub-06,Early practice,8-3-9,0.133333,10.194895,-8.961299,6.0,0.833333,75.940503,3.608475,w8l3,9
12005,sub-06,Late discovery,8-3-9,0.133333,10.194895,-8.961299,6.0,0.833333,75.943115,5.003390,w8l3,9


In [31]:
df_human = df[df['subject'].str.startswith('sub-')].copy()
df_human['phase'] = df_human['learning_phase'].apply(lambda x: x.split(' ')[1])
df_human['count_phase'] = df_human.groupby(['phase', 'scene_full_name', 'subject'])['count'].transform('sum')
mask_drope = df_human['count_phase'] < 10
scene_to_drope = df_human.loc[mask_drope, 'scene_full_name'].unique()
mask_drope = df['scene_full_name'].isin(scene_to_drope)
df = df[~mask_drope].copy()
print(df['scene_full_name'].nunique())
df

40


,subject,learning_phase,scene_full_name,delta_clr_tot,delta_spd_tot,delta_MAD_tot,count,cleared,speed,MAD_mean,level_full_name,scene,sub_type
3364,im_sub-01,sub-01_epoch=0-step=2000,2-3-0,0.000000,0.000000,NaN,58.0,0.017241,1.516458,NaN,w2l3,0,imi
3944,im_sub-01,sub-01_epoch=0-step=3500,2-3-0,0.000000,0.000000,NaN,58.0,0.017241,1.516458,NaN,w2l3,0,imi
3654,im_sub-01,sub-01_epoch=0-step=500,2-3-0,0.000000,0.000000,NaN,58.0,0.017241,1.516458,NaN,w2l3,0,imi
4234,im_sub-01,sub-01_epoch=0-step=5000,2-3-0,0.000000,0.000000,NaN,58.0,1.000000,67.660386,NaN,w2l3,0,imi
3074,im_sub-01,sub-01_epoch=0-step=6500,2-3-0,0.000000,0.000000,NaN,58.0,0.017241,1.516458,NaN,w2l3,0,imi
...,...,...,...,...,...,...,...,...,...,...,...,...,...
11744,sub-05,Late practice,8-3-3,-0.065217,-34.268135,-0.136364,23.0,0.434783,45.401466,32.069091,w8l3,3,hum
12324,sub-06,Early discovery,8-3-3,-0.037037,18.574075,9.112727,27.0,0.592593,61.047781,8.265455,w8l3,3,hum
12614,sub-06,Early practice,8-3-3,-0.037037,18.574075,9.112727,26.0,0.576923,80.552702,19.150909,w8l3,3,hum
12034,sub-06,Late discovery,8-3-3,-0.037037,18.574075,9.112727,24.0,0.750000,105.517079,17.774545,w8l3,3,hum


In [32]:
def get_sub_type(row):
    if row['subject'] == 'ppo':
        return 'ppo'
    elif row['subject'].startswith('im'):
        return 'imi'
    else:
        return 'hum'

df['sub_type'] = df.apply(get_sub_type, axis=1)

df_sub_type = (
    df
    .groupby(['scene_full_name', 'sub_type'])[
        ['delta_clr_tot', 'delta_spd_tot', 'MAD_mean']
    ]
    .mean()
    .reset_index()
)
df_sub_type.head(20)

,scene_full_name,sub_type,delta_clr_tot,delta_spd_tot,MAD_mean
0,2-3-0,hum,0.016304,5.239464,3.238839
1,2-3-0,imi,0.034054,7.387870,NaN
2,2-3-0,ppo,0.000000,11.340051,0.787968
3,2-3-1,hum,0.028571,10.732344,0.577731
4,2-3-1,imi,-0.014404,0.601661,NaN
5,2-3-1,ppo,0.021583,21.424895,8.434056
6,2-3-2,hum,0.134012,21.666948,3.190002
7,2-3-2,imi,-0.001231,0.464718,NaN
8,2-3-2,ppo,0.302920,50.932550,11.550614
9,3-2-0,hum,-0.002149,1.750571,3.512676


In [9]:
thres_clr = 15
thres_spd = 15
thres_MAD = 15
thres = 15

In [34]:
metrics = ["delta_clr_tot", "delta_spd_tot", "MAD_mean"]
thres = 15 # top-n / bottom-n

dfs = []

for metric in metrics:
    # top
    top_df = (
        df_sub_type.groupby("sub_type", group_keys=False)
        .apply(lambda x: x.nlargest(thres, metric))
        .assign(selected_by=f"{metric}_top")
    )
    dfs.append(top_df)
    
    # bottom
    bottom_df = (
        df_sub_type.groupby("sub_type", group_keys=False)
        .apply(lambda x: x.nsmallest(thres, metric))
        .assign(selected_by=f"{metric}_bottom")
    )
    dfs.append(bottom_df)

df_select = pd.concat(dfs, ignore_index=True)

# Compter combien de sub_type par scène
scene_counts = df_select.groupby("scene_full_name")["sub_type"].nunique()

# Scènes apparaissant dans >1 sub_type
scenes_intersection = scene_counts[scene_counts > 1].index

df_intersection = df_select[df_select["scene_full_name"].isin(scenes_intersection)]

/tmp/ipykernel_47150/1984758985.py:10: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: x.nlargest(thres, metric))
/tmp/ipykernel_47150/1984758985.py:18: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: x.nsmallest(thres, metric))
/tmp/ipykernel_47150/1984758985.py:10: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future versi

In [35]:
df_exploded = df_intersection.explode("sub_type").explode("selected_by")

df_pivot = (
    df_intersection
    .groupby(["scene_full_name", "selected_by"])["sub_type"]
    .unique()  # retourne un array de sub_type unique
    .apply(list)  # si tu veux convertir en liste
    .unstack(fill_value=[])
)

df_pivot

selected_by,MAD_mean_bottom,MAD_mean_top,delta_clr_tot_bottom,delta_clr_tot_top,delta_spd_tot_bottom,delta_spd_tot_top
scene_full_name,,,,,,
2-3-0,"[hum, imi, ppo]",[imi],[ppo],[],[],[imi]
2-3-1,"[hum, imi]",[imi],[imi],[],[],"[hum, ppo]"
2-3-2,"[hum, imi]",[imi],[imi],"[hum, ppo]",[],"[hum, ppo]"
3-2-0,"[hum, imi, ppo]",[imi],"[hum, imi]",[],"[hum, imi]",[ppo]
3-2-1,"[hum, imi]",[imi],[hum],"[imi, ppo]",[hum],"[imi, ppo]"
3-2-2,"[imi, ppo]",[imi],[hum],"[imi, ppo]",[hum],"[imi, ppo]"
3-2-3,"[imi, ppo]",[imi],[],"[hum, imi]",[ppo],"[hum, imi]"
4-3-0,"[hum, imi, ppo]",[imi],"[hum, imi, ppo]",[],"[hum, imi]",[ppo]
4-3-1,"[imi, ppo]",[imi],[],"[hum, imi, ppo]","[imi, ppo]",[]


In [36]:
print(f"Nombre de scènes sélectionnées: {df_select['scene_full_name'].nunique()}")

Nombre de scènes sélectionnées: 40
